In [ ]:
import os
import pandas as pd

# Directory containing the CSV files
data_dir = '../johns_hopkins_github_data_pull/covid_reports_2020'

# List all CSV files in the directory
csv_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]

# Initialize an empty list to hold individual DataFrames
dataframes = []

# Load each CSV file into a DataFrame
for file in csv_files:
    file_path = os.path.join(data_dir, file)
    df = pd.read_csv(file_path)
    # Extract the date from the filename and create a new column for it
    date = file.split('.')[0]  # Assumes filename is of the form 'MM-DD-YYYY.csv'
    df['Date'] = pd.to_datetime(date, format='%m-%d-%Y')
    # Append the DataFrame to the list
    dataframes.append(df)

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

# Filter for 'US' country and select necessary columns
us_data = combined_df[combined_df['Country_Region'] == 'US']

# Pivot the DataFrame to get 'Confirmed' cases as columns
pivot_df = us_data.pivot_table(index=['Province_State', 'Lat', 'Long'], columns='Date', values='Confirmed', aggfunc='sum')

# Flatten the columns and rename them to include the date in the format 'Confirmed MM-DD-YYYY'
pivot_df.columns = [f'Confirmed {date.strftime("%m-%d-%Y")}' for date in pivot_df.columns]

# Reset index to include 'Province_State', 'Lat', and 'Long' as columns
final_df = pivot_df.reset_index()

# Display the final DataFrame
print(final_df)

# Save the final DataFrame to a CSV file
final_df.to_csv('consolidated_covid_data_us_with_lat_long.csv', index=False)
